In [115]:
import pandas as pd
import numpy as np

In [116]:
df = pd.read_csv("DonneesMeteo.txt",delim_whitespace=True, low_memory=False, skiprows=1)

In [128]:
import datetime
#time = temp = pd.DatetimeIndex(df1['DT_local'])
date = pd.DatetimeIndex(pd.to_datetime(df['Date'], format="%Y%m%d"))
df['MONTH'] = date.month
df['DAY'] = date.day
df['YEAR'] = date.year
#Version arrondie à la minute de la ligne du dessous : num = pd.to_datetime(pd.Series.round(df['hh(UTC)']*3600)*1000000000)
hh = pd.DatetimeIndex(pd.to_datetime(df['hh(UTC)']*3600*1000000000))
df['HOUR'] = hh.hour
df['MINUTE'] = hh.minute

On va à présent se concentrer sur la prédiction de :
    - puissance
Selon
    - T
    - vent
    - skycover
    - precipitation

In [118]:
#df.columns.values.tolist()

# RandomForest

In [123]:
#Paramètres
colsX = ['DAY',
    'Date',
 'hh(UTC)',
 'Tp1(C)',
 'Tp2(C)',
 'IrrPOA(W/m2)',
 'Gz2(W/m2)',
 'Drz2(W/m2)',
 'Dfz2(W/m2)',
 'IRz2(W/m2)',
 'T(C)',
 'WS(m/s)',
 'WD(deg)',
 'mpGz1(W/m2)',
 'IRz1dn(W/m2)',
 'IRz1up(W/m2)',
 'mpT(C)',
 'mpWS(m/s)',
 'mpWD(deg)',
 'SZA(deg)',
 'SAA(deg)']
#colsX = ["DayPart", "T(C)", ]
colsY = ["P1(W)", "P2(W)"]
#print(cols)

In [124]:
import matplotlib.pyplot as plt

#preparer (nan, découpe)
X,y = prepare(df,colsX,colsY)
#Découper
X_train, X_cv, X_test, y_train, y_cv, y_test = decoupe(X,y)
#scaler
X_train, X_cv, X_test, y_train, y_cv, y_test = featureScale(X_train, X_cv, X_test, y_train, y_cv, y_test)
#

In [127]:
# Create linear regression object
from sklearn import ensemble
regr = sklearn.ensemble.RandomForestRegressor()
# Train the model using the training sets
regr.fit(X_train, y_train)
# The mean square error
print("Residual sum of squares: %.2f" % np.mean((regr.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))

Residual sum of squares: 0.03
Variance score: 0.97
-1.8050023185
-1.56185620589
-1.44028314959
-1.31871009329
-1.19713703698
-1.07556398068
-0.953990924375
-0.832417868071
-0.710844811767
-0.589271755463
-0.467698699159
-0.346125642856
-0.224552586552
-0.102979530248
0.0185935260558
0.14016658236
0.261739638663
0.383312694967
0.504885751271
0.626458807575
0.748031863879
0.869604920182
0.991177976486
1.11275103279
1.23432408909
1.3558971454
1.4774702017
1.59904325801


In [ ]:
plotterIterations(X_test,2,1,y_test,0)

In [126]:
# Plot outputs 2D
import matplotlib.cm as cm

def plotterIterations(X,nX,nIt,y,ny) :
    #On se ramène à des journées
    EltIteration = X[:,nIt]
    Iterations = np.unique(EltIteration)

    #Gérer plusieurs couleurs
    colors = cm.rainbow(np.linspace(0, 1, len(Iterations)))

    for k,c in zip(Iterations,colors)  :
        print(k)
        #On prend que le jour
        condition = EltIteration == k #entre 60 et 153
        xPlot = np.extract(condition, X[:,nX])
        yPlot = np.extract(condition, y[:,ny])
        plt.scatter(xPlot, yPlot, color=c,linewidth=2)

    #plt.plot(X_test, regr.predict(X_test), color='blue',linewidth=3)
    plt.xticks(())
    plt.yticks(())

    #plt.xlabel(colsX[nX])
    #plt.ylabel(colsY[nY])
    plt.title('')

    plt.show()

In [129]:
import sklearn.cross_validation as cv
import sklearn.preprocessing as pp #pour les scaling

def rms(Y1,Y2):
    return np.sqrt(np.mean((Y1 - Y2)**2))
    
def randomforestcross(X,Y):
    X_train, X_1, y_train, y_1 = cv.train_test_split(X, Y, test_size=0.4, random_state=0)
    X_cr, X_test, y_cr, y_test = cv.train_test_split(X_1, y_1, test_size=0.5, random_state=0)
    RMS=np.zeros((100,2))
    
    for i in range(1,25):
        y_=randomforest(i,X_train,y_train,X_cr)
        r=rms(y_,y_cr)
        RMS[i,0]=i
        RMS[i,1]=r
    plt.scatter(RMS[:,0],RMS[:,1])
    plt.show()
    L = [X_train, y_train, X_test, y_test]
    return L 

def randomforest(n,X,Y,X_t):
    clf= RandomForestRegressor(n_estimators=n)
    clf.fit(X,Y)
    y_=clf.predict(X_t)
    return y_

def gradientcross(X,Y):
    X_train, X_cv, X_test, y_train, y_cv, y_test = decoupe(X,y)
    RMS=np.zeros((250,2))
    
    for i in range(1,10):
        i = i * 10 + 100
        y_=gradient(i,X_train,y_train,X_cr)
        r=rms(y_,y_cr)
        RMS[i,0]=i
        RMS[i,1]=r
    plt.scatter(RMS[:,0],RMS[:,1])
    plt.show()
    L = [X_train, y_train, X_test, y_test]
    return L 
def gradient(n,X,Y,X_t):
    clf= ensemble.GradientBoostingRegressor(n_estimators=n)
    clf.fit(X,Y)
    y_=clf.predict(X_t)
    return y_


def prepare(df,colsX,colsY):
    #Split the treated data between X and y
    Z = vireNan(df[colsX+colsY])
    deb = 0
    sep = len(colsX)#separteur entre les X et les Y
    fin = sep+len(colsY)
    X=Z[:,deb:sep]
    y=Z[:,sep:fin]
    return X,y

import numpy as np
def vireNan(Z) :
    # Load the dataset
    Z = np.array(Z).astype(np.float)

    #Remove nan and infinite values
    masknan = ~np.any(np.isnan(Z), axis=1)
    Z = Z[masknan]
    maskfin = np.any(np.isfinite(Z), axis=1)
    Z = Z[maskfin]
    return Z

def decoupe(X,y) :
    X_train, X_1, y_train, y_1 = cv.train_test_split(X, y, test_size=0.4, random_state=0)
    X_cv, X_test, y_cv, y_test = cv.train_test_split(X_1, y_1, test_size=0.1, random_state=0)
    return X_train, X_cv, X_test, y_train, y_cv, y_test

#variable globales
fsX = pp.StandardScaler()
fsy = pp.StandardScaler()
def featureScale(X_train, X_cv, X_test, y_train, y_cv, y_test) :
    #feature scaling inti
    fsX.fit(X_train)
    X_train = fsX.transform(X_train)
    X_cv = fsX.transform(X_cv)
    X_test = fsX.transform(X_test)
    
    fsy.fit(y_train)
    y_train = fsy.transform(y_train)
    y_cv = fsy.transform(y_cv)
    y_test = fsy.transform(y_test)
    
    return X_train, X_cv, X_test, y_train, y_cv, y_test
